# ANALYSIS 3

# FINDING THE NEAREST 10 RESTAURANTS TO A GIVEN ADDRESS

### ALL REQUIRED IMPORTS

In [4]:
import string
import csv, sys
from pandas import Series, DataFrame
import pandas as pd
import os
import glob
from os.path import basename
import fnmatch
import json
import nltk
from pprint import pprint
from geopy.geocoders import Nominatim
from geopy.distance import vincenty

#### GETTING THE SYSTEM PATH

In [84]:
directory = os.path.dirname(os.getcwd()) 
path = directory+'/data/'
path
directory

'/Users/sonalichaudhari/Desktop/final'

#### TAKING USER INPUT FOR ADDRESS  

In [21]:
print('STREET NAME AND CITY NAME COUNTRY MANDATORY')
address1 = input("Enter Address: ")
print (address1) 

STREET NAME AND CITY NAME COUNTRY MANDATORY
Enter Address: smith street boston
smith street boston


#### Retrieving the complete address with latitude and longitude

In [22]:
# Getting the complete address 

geolocator = Nominatim()
location = geolocator.geocode(address1) #Passing the address which stores user input
print(location.address)
print((location.latitude, location.longitude))

Smith Street, Roxbury Crossing, Fenway, Boston, Suffolk County, Massachusetts, 02120, United States of America
(42.3349341, -71.1023921)


In [24]:
# Storing the source latitude and longitude 

s_lat = location.latitude
s_lon = location.longitude

####  STORING OF INTERMEDIATE DATA

In [67]:
DISTANCE={} # to store the  restaurant ids and their distances whose distance is less than 25 miles
temp = [] # to store th data about all the restaurants whose distance is less than 25 miles
for subdir, dirs, files in os.walk(path):
        for file in files:           
            if file.endswith('.json'): # Retrieving only the JSON files
                reqpath = subdir+'/'+file # Path of evry JSON file is stored in reqpath
                
                with open(reqpath) as json_file:
                    g = json.load(json_file)
                    des_lat = g['restaurant']['location']['latitude']  # Destination latitude value
                    des_log = g['restaurant']['location']['longitude'] # Destination longitude value
                    source =(s_lat,s_lon)
                    destination =(des_lat,des_log)
# Finding the distance between the source and destinaltion 
                    d = vincenty(source, destination).miles
                    if d<=25:    # Filtering for distance less than 25(miles)
                        temp.append(g)
                        dd= round(d,4)
                        DISTANCE[g['restaurant']['R']['res_id']]= dd
                

In [68]:
# Sorting in ascending order of distance and getting the top 10
S = sorted(DISTANCE.items(), key=lambda x: x[1], reverse=False)[:10]

In [92]:
# Storing the restaurant id of top 10 obtained above in RES_ID list
RES_ID=[]
for k,v in S:
    RES_ID.append(k)
#RES_ID

In [76]:
# Storing the data of restaurant of top 10 obtained above in RES_ID list
RES=[]

for e in temp:
    if (e['restaurant']['R']['res_id'] in RES_ID) == True:
        RES.append(e)
        

#### STORING THE TEMP LIST DATA IN A CSV

In [109]:
with open(directory+'/analysis/ana_3/ANA_3_Part1.csv', 'w') as csvfile:    # Creating CSV
    fieldnames = ['RES_ID', 'Name','CUISINES','USER RATING','VOTES',\
                 'RATING TEXT','ADDRESS','AVG COST for TWO','DISTANCE']#Naming the title Row of the CSV
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for j in range(len(RES)-1):
        for k,v in S: # if restaurant id is present in the list of top 10 nearest restaurants list
            if k == RES[j]['restaurant']['R']['res_id']:
                r = RES[j]['restaurant']['R']['res_id']
                n = RES[j]['restaurant']['name']
                cu= RES[j]['restaurant']['cuisines']
                u = RES[j]['restaurant']['user_rating']['aggregate_rating']
                vo = RES[j]['restaurant']['user_rating']['votes']
                rt= RES[j]['restaurant']['user_rating']['rating_text']
                add= RES[j]['restaurant']['location']['address']
                avg=str(RES[j]['restaurant']['average_cost_for_two'])+temp[j]['restaurant']['currency']
                dis = str(v)+' miles'
                # appending the rows to the csv
                writer.writerow({'RES_ID':r, 'Name':n,'CUISINES':cu,'USER RATING':u,\
                                 'VOTES':vo,'RATING TEXT':rt,'ADDRESS':add,'AVG COST for TWO':avg,'DISTANCE':dis}) 

#### CONVERTING THE CSV INTO DATAFRAME

In [111]:
t = pd.read_csv(directory+'/analysis/ana_3/ANA_3_Part1.csv').reset_index(0,drop=True)
t.head()

,RES_ID,Name,CUISINES,USER RATING,VOTES,RATING TEXT,ADDRESS,AVG COST for TWO,DISTANCE
0,16794052,Abe & Louie's,"Steak, American",4.3,680,Very Good,"793 Boylston Street, Boston, MA 02116",70$,1.4441 miles
1,16794268,Atlantic Fish Co.,Seafood,4.5,670,Excellent,"761 Boylston Street, Boston, MA 02116",40$,1.4658 miles
2,16795720,Eastern Standard,"American, Seafood",4.5,622,Excellent,"528 Commonwealth Avenue, Boston, MA 02215",70$,1.0015 miles
3,16795900,Flour Bakery + Cafe,"Bakery, Cafe",4.7,854,Excellent,"1595 Washington Street, Boston, MA 02118",10$,1.4458 miles
4,16795961,Fugakyu,"Asian, Japanese, Sushi",4.3,968,Very Good,"1280 Beacon St, Brookline, MA 02446",150$,1.0534 miles


In [ ]:
place_data = {}
for place in place_frame['stop_name']:
    r = requests.get('http://nominatim.openstreetmap.org/?q='+place+'&format=json')
    place_data[place] = r.json()